<a href="https://colab.research.google.com/github/Umayanga12/Sentiment-Analysis-NLP/blob/main/Sentiment_Analysis_using_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import nltk
import string
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data_set_url='/content/drive/MyDrive/Dataset/Tweets.csv' #Path to the data set
data_set = pd.read_csv(data_set_url)
data_set.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [5]:
data_set=data_set.drop(['tweet_id','airline_sentiment_confidence','negativereason','negativereason_confidence','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],axis=1)

In [6]:
data_set.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [7]:
nltk.download('punkt')
ps = PorterStemmer()

y = []

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)
    text = nltk.word_tokenize(text)

    for i in text:
      if i not in stopwords.words('english'):
        y.append(i)
      text = y[:]
      y.clear()

    for i in text:
        y.append(ps.stem(i))

    return" ".join(y)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
text=data_set['text']

for i in text:
  clean_text(i)

In [9]:
def add_cleaned_text_column(df):
    data_set['cleaned_text'] = data_set['text'].apply(clean_text)
    return data_set

if __name__ == "__main__":
    data_set = add_cleaned_text_column(data_set)
    print(data_set)

      airline_sentiment                                               text  \
0               neutral                @VirginAmerica What @dhepburn said.   
1              positive  @VirginAmerica plus you've added commercials t...   
2               neutral  @VirginAmerica I didn't today... Must mean I n...   
3              negative  @VirginAmerica it's really aggressive to blast...   
4              negative  @VirginAmerica and it's a really big bad thing...   
...                 ...                                                ...   
14635          positive  @AmericanAir thank you we got on a different f...   
14636          negative  @AmericanAir leaving over 20 minutes Late Flig...   
14637           neutral  @AmericanAir Please bring American Airlines to...   
14638          negative  @AmericanAir you have my money, you change my ...   
14639           neutral  @AmericanAir we have 8 ppl so we need 2 know h...   

       cleaned_text  
0                 .  
1                 .

In [10]:
text_cleaned = data_set['cleaned_text']
tfidf_vectorizer = TfidfVectorizer(max_features=3000)
tfidf_vectorizer.fit(text_cleaned)
X = tfidf_vectorizer.transform(text_cleaned)
X = X.toarray()
Y = np.array(data_set['airline_sentiment'])

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=.2,random_state=2)

In [12]:
from sklearn.naive_bayes import MultinomialNB
import sklearn.metrics
clf = MultinomialNB()
clf.fit(X, Y)
predictions = clf.predict(x_test)


In [15]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(X)
print(y_pred)

['negative' 'negative' 'negative' ... 'neutral' 'negative' 'negative']
